<a href="https://colab.research.google.com/github/ShaharBar99/Snake_Cloud_Project/blob/main/Cloud_Snake_HW2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# --- BLOCK: Screen 2 - FIXED Data Order (Sync Graph & Button) ---

# !pip install ipywidgets requests matplotlib
import ipywidgets as widgets
import requests
import matplotlib.pyplot as plt
from IPython.display import display, clear_output
import datetime

# --- A. Configuration ---
BASE_URL = "https://server-cloud-v645.onrender.com/"
current_state = {
    'feed': 'temperature',
    'limit': 10
}

# --- B. Styling (CSS) ---
style_css = """
<style>
    @import url('https://fonts.googleapis.com/css2?family=Roboto:wght@400;700&display=swap');

    .screen-container {
        font-family: 'Roboto', sans-serif;
        background-color: #f4f6f7;
        padding: 15px;
        border-radius: 15px;
        box-shadow: 0 4px 15px rgba(0,0,0,0.1);
        width: 98%;
        border: 1px solid #dcdcdc;
    }
    .header-text {
        color: #2c3e50;
        text-align: center;
        margin-bottom: 20px;
        font-weight: 700;
        font-size: 28px;
    }
    .widget-label { font-size: 20px !important; color: #333 !important; }
    .widget-html { font-size: 20px !important; }
    .widget-text { font-size: 20px !important; }
    .widget-button {
        font-size: 20px !important;
        border-radius: 12px !important;
        font-weight: bold !important;
    }
    input {
        font-size: 20px !important;
        text-align: center !important;
        padding: 5px !important;
    }
</style>
"""

# --- C. Data Fetching ---

def fetch_latest_value(feed_name):
    """Fetches single latest value."""
    try:
        response = requests.get(f"{BASE_URL}/history", params={"feed": feed_name, "limit": 1}, timeout=3)
        data = response.json()
        if "data" in data and len(data["data"]) > 0:
            return str(data["data"][0]["value"])
    except:
        return "--"
    return "--"

def fetch_history_data(feed_name, limit_count):
    """Fetches history list."""
    try:
        response = requests.get(f"{BASE_URL}/history", params={"feed": feed_name, "limit": limit_count}, timeout=5)
        data = response.json()
        if "data" in data:
            vals = [float(x['value']) for x in data['data']]

            # --- FIX IS HERE ---
            # We MUST reverse the list.
            # Server gives: [Newest, ..., Oldest]
            # Graph needs:  [Oldest, ..., Newest] (Left to Right)
            vals.reverse()

            return vals
    except:
        return []
    return []

# --- D. Widgets ---

header = widgets.HTML(f"{style_css}<div class='screen-container'><div class='header-text'>📡 IoT Sensor Monitor</div>")
footer = widgets.HTML("</div>")

btn_temp = widgets.Button(description="Temp: --°C", button_style='danger', layout=widgets.Layout(width='32%', height='80px'), icon='thermometer-half')
btn_humid = widgets.Button(description="Humid: --%", button_style='info', layout=widgets.Layout(width='32%', height='80px'), icon='tint')
btn_soil = widgets.Button(description="Soil: --%", button_style='success', layout=widgets.Layout(width='32%', height='80px'), icon='leaf')

lbl_limit = widgets.HTML(
    value="<div style='background-color: white; padding: 8px 15px; border-radius: 10px; border: 1px solid #ccc; font-weight: bold;'>Graph History(limit):</div>",
    layout=widgets.Layout(margin='0 10px 0 0')
)

input_limit = widgets.IntText(value=10, layout=widgets.Layout(width='80px', height='30px'))

out_graph = widgets.Output(
    layout=widgets.Layout(
        width='100%',
        display='flex',
        justify_content='center',
        align_items='center',
        margin='20px 0 0 0'
    )
)

# --- E. Logic ---

def refresh_all_data():
    """Main Update Function"""
    btn_refresh.icon = 'spin fa-spinner'
    btn_refresh.description = ' Loading...'

    # 1. Update Buttons
    btn_temp.description = f" Temp: {fetch_latest_value('temperature')}°C"
    btn_humid.description = f" Humid: {fetch_latest_value('humidity')}%"
    btn_soil.description = f" Soil: {fetch_latest_value('soil')}%"

    # 2. Update Graph
    feed = current_state['feed']
    limit = current_state['limit']

    with out_graph:
        clear_output(wait=True)
        values = fetch_history_data(feed, limit)

        if not values:
            print("⚠️ Server sleeping or no data. Try again.")
        else:
            plt.figure(figsize=(10, 4))
            plt.style.use('seaborn-v0_8-whitegrid')

            c_map = {'temperature': '#e74c3c', 'humidity': '#3498db', 'soil': '#2ecc71'}
            c = c_map.get(feed, '#333')

            plt.plot(values, marker='o', color=c, linewidth=3, label=feed.capitalize())
            plt.fill_between(range(len(values)), values, color=c, alpha=0.1)

            # Fonts
            plt.title(f"Live Trend: {feed.capitalize()} (Last {limit})", fontsize=24, fontweight='bold', pad=20)
            plt.xlabel("Sample Order (Oldest → Newest)", fontsize=18)
            plt.ylabel("Sensor Value", fontsize=18)
            plt.tick_params(axis='both', which='major', labelsize=16)

            plt.legend(fontsize=16)
            plt.tight_layout()
            plt.show()

    btn_refresh.icon = 'refresh'
    btn_refresh.description = ' Sync'

def highlight_button(active_btn):
    for b in [btn_temp, btn_humid, btn_soil]:
        b.layout.border = None
    active_btn.layout.border = '4px solid #555'

# --- F. Event Handlers ---

def on_temp_click(b):
    current_state['feed'] = 'temperature'
    highlight_button(btn_temp)
    refresh_all_data()

def on_humid_click(b):
    current_state['feed'] = 'humidity'
    highlight_button(btn_humid)
    refresh_all_data()

def on_soil_click(b):
    current_state['feed'] = 'soil'
    highlight_button(btn_soil)
    refresh_all_data()

def on_refresh_click(b):
    refresh_all_data()

def on_limit_change(change):
    val = change['new']
    if val < 10: input_limit.value = 10
    elif val > 50: input_limit.value = 50
    else:
        current_state['limit'] = val
        refresh_all_data()

# Connect
btn_temp.on_click(on_temp_click)
btn_humid.on_click(on_humid_click)
btn_soil.on_click(on_soil_click)
input_limit.observe(on_limit_change, names='value')

# --- G. Run ---
top_row = widgets.HBox([btn_temp, btn_humid, btn_soil], layout=widgets.Layout(justify_content='space-between', margin='0 0 20px 0'))
ctrl_row = widgets.HBox([lbl_limit, input_limit], layout=widgets.Layout(margin='0 0 20px 0', align_items='center'))

ui = widgets.VBox([header, top_row, ctrl_row, out_graph, footer])

display(ui)
highlight_button(btn_temp)
refresh_all_data()

In [ ]:
# --- BLOCK 2: Screen 4 - Visual Dashboard ---

# 1. Imports
import ipywidgets as widgets
from IPython.display import display, HTML
import matplotlib.pyplot as plt
import numpy as np # Used for generating graph data

# --- A. Setup & Styles ---
# CSS for the status cards
card_style = """
<style>
.card {
    background-color: #f8f9fa;
    border-radius: 10px;
    padding: 15px;
    text-align: center;
    box-shadow: 0 4px 8px 0 rgba(0,0,0,0.2);
    margin: 5px;
    font-family: Arial, sans-serif;
}
.metric-value { font-size: 24px; font-weight: bold; color: #333; }
.metric-label { font-size: 14px; color: #777; }
</style>
"""

# --- B. Dashboard Components ---

# 1. Header
dashboard_header = widgets.HTML(
    value=f"{card_style}<h2 style='text-align: center; color: #196F3D;'>🌿 Smart Garden Dashboard</h2>"
)

# 2. Status Indicators (HTML Widgets)
# These act as "Cards" showing current values
temp_card = widgets.HTML(value="<div class='card'><div class='metric-value'>-- °C</div><div class='metric-label'>Temperature</div></div>")
humid_card = widgets.HTML(value="<div class='card'><div class='metric-value'>-- %</div><div class='metric-label'>Humidity</div></div>")
soil_card = widgets.HTML(value="<div class='card'><div class='metric-value'>-- %</div><div class='metric-label'>Soil Moisture</div></div>")

# 3. Main Status Message
status_message = widgets.HTML(
    value="<div style='background-color: #ddd; padding: 10px; text-align: center; border-radius: 5px;'>Waiting for data...</div>"
)

# 4. Refresh Button
btn_refresh = widgets.Button(
    description='🔄 Refresh Data',
    button_style='primary',
    layout=widgets.Layout(width='100%'),
    icon='chart-line'
)

# 5. Graph Output Area
graph_output = widgets.Output()

# --- C. Logic & Visualization Functions ---

def get_status_color(moisture):
    """
    Determines status color based on soil moisture.
    Returns a HTML color string.
    """
    if moisture < 30:
        return "#E74C3C" # Red (Critical)
    elif moisture < 50:
        return "#F1C40F" # Yellow (Warning)
    else:
        return "#2ECC71" # Green (Good)

def update_dashboard(b):
    """
    Fetches data (simulated) and updates all visual elements.
    """
    # 1. Simulate fetching latest data from Cloud/DB
    # In real app: data = firebase.get('/sensor_data', None)
    current_temp = np.random.randint(20, 32)
    current_humid = np.random.randint(40, 70)
    current_soil = np.random.randint(20, 90)

    # 2. Update Cards HTML
    temp_card.value = f"<div class='card'><div class='metric-value'>{current_temp}°C</div><div class='metric-label'>Temperature</div></div>"
    humid_card.value = f"<div class='card'><div class='metric-value'>{current_humid}%</div><div class='metric-label'>Humidity</div></div>"
    soil_card.value = f"<div class='card'><div class='metric-value'>{current_soil}%</div><div class='metric-label'>Soil Moisture</div></div>"

    # 3. Logic for Plant Health Status
    color = get_status_color(current_soil)
    status_text = "HEALTHY" if current_soil > 50 else ("NEEDS WATER" if current_soil > 30 else "CRITICAL")

    status_message.value = f"""
    <div style='background-color: {color}; color: white; padding: 15px; text-align: center; border-radius: 5px; font-weight: bold; font-size: 18px;'>
        STATUS: {status_text}
    </div>
    """

    # 4. Draw Graph (Simulating history of last 10 readings)
    with graph_output:
        clear_output(wait=True) # Clear previous graph
        # Create dummy history data
        x = ['10:00', '10:05', '10:10', '10:15', '10:20']
        y_soil = np.random.randint(30, 80, size=5)
        y_temp = np.random.randint(22, 28, size=5)

        plt.figure(figsize=(6, 3))
        plt.plot(x, y_soil, marker='o', label='Soil Moisture (%)', color='green')
        plt.plot(x, y_temp, marker='s', label='Temp (°C)', color='orange', linestyle='--')
        plt.title('Last 20 Minutes Trends')
        plt.legend()
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.show()

# --- D. Layout Assembly ---
# Bind event
btn_refresh.on_click(update_dashboard)

# Top row: 3 Cards
cards_row = widgets.HBox([temp_card, humid_card, soil_card], layout=widgets.Layout(justify_content='space-between'))

# Main Structure
dashboard = widgets.VBox(
    [dashboard_header, status_message, cards_row, btn_refresh, graph_output],
    layout=widgets.Layout(width='600px', border='1px solid #ccc', padding='20px')
)

# Initialize with data once on load
update_dashboard(None)

display(dashboard)

In [ ]:
# --- BLOCK 2: Screen 4 - Plant Dashboard (Visual Upgrade) ---

import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
import matplotlib.pyplot as plt
import numpy as np

# --- 1. Dashboard CSS ---
dash_style = """
<style>
    @import url('https://fonts.googleapis.com/css2?family=Montserrat:wght@400;600&display=swap');

    .dash-wrapper {
        font-family: 'Montserrat', sans-serif;
        background-color: #ecf0f1;
        padding: 20px;
        border-radius: 15px;
        width: 600px;
        border: 1px solid #bdc3c7;
        margin: 0 auto;
    }
    .dash-header {
        display: flex;
        align-items: center;
        margin-bottom: 20px;
        background-color: white;
        padding: 15px;
        border-radius: 10px;
        box-shadow: 0 2px 5px rgba(0,0,0,0.05);
    }
    .header-icon { font-size: 30px; margin-right: 15px; }
    .header-text { font-size: 20px; font-weight: 600; color: #27ae60; }

    /* KPI Cards Container */
    .kpi-container {
        display: flex;
        justify-content: space-between;
        margin-bottom: 20px;
    }
    .kpi-card {
        background: white;
        width: 30%;
        padding: 15px;
        border-radius: 12px;
        text-align: center;
        box-shadow: 0 4px 6px rgba(0,0,0,0.05);
        transition: transform 0.2s;
    }
    .kpi-card:hover { transform: translateY(-3px); }
    .kpi-value { font-size: 28px; font-weight: bold; color: #2c3e50; }
    .kpi-label { font-size: 11px; color: #95a5a6; text-transform: uppercase; letter-spacing: 1px; margin-top: 5px; }

    /* Status Banner */
    .status-banner {
        padding: 12px;
        border-radius: 8px;
        text-align: center;
        font-weight: bold;
        color: white;
        margin-bottom: 20px;
        box-shadow: 0 2px 4px rgba(0,0,0,0.1);
    }
</style>
"""

# --- 2. Widgets & Components ---

header_w = widgets.HTML(f"{dash_style}<div class='dash-wrapper'><div class='dash-header'><span class='header-icon'>🌿</span><span class='header-text'>Live Plant Analytics</span></div>")

# We will generate the KPI cards dynamically using HTML strings
kpi_row = widgets.HTML() # Placeholder for the cards
status_row = widgets.HTML() # Placeholder for status
graph_out = widgets.Output()

btn_refresh = widgets.Button(
    description=' Sync Live Data',
    icon='refresh',
    button_style='primary',
    layout=widgets.Layout(width='100%', height='45px')
)

footer_w = widgets.HTML("</div>") # Close dash-wrapper

# --- 3. Logic & Visualization ---

def generate_kpi_html(temp, humid, soil):
    return f"""
    <div class='kpi-container'>
        <div class='kpi-card' style='border-bottom: 4px solid #e74c3c'>
            <div class='kpi-value'>{temp}°</div>
            <div class='kpi-label'>Temperature</div>
        </div>
        <div class='kpi-card' style='border-bottom: 4px solid #3498db'>
            <div class='kpi-value'>{humid}%</div>
            <div class='kpi-label'>Air Humidity</div>
        </div>
        <div class='kpi-card' style='border-bottom: 4px solid #2ecc71'>
            <div class='kpi-value'>{soil}%</div>
            <div class='kpi-label'>Soil Moist.</div>
        </div>
    </div>
    """

def update_ui(b):
    # 1. Simulate Data
    t = np.random.randint(22, 30)
    h = np.random.randint(45, 65)
    s = np.random.randint(20, 95)

    # 2. Update HTML Widgets
    kpi_row.value = generate_kpi_html(t, h, s)

    # Determine Status
    if s < 30:
        bg, txt = "#c0392b", "CRITICAL: WATER NEEDED" # Red
    elif s < 50:
        bg, txt = "#f39c12", "WARNING: SOIL DRYING" # Orange
    else:
        bg, txt = "#27ae60", "SYSTEM OPTIMAL" # Green

    status_row.value = f"<div class='status-banner' style='background-color: {bg};'>{txt}</div>"

    # 3. Update Graph (Matplotlib with style)
    with graph_out:
        clear_output(wait=True)
        plt.style.use('seaborn-v0_8-whitegrid') # Modern clean style
        fig, ax = plt.subplots(figsize=(8, 3.5))

        # Dummy history
        x = ['10:00', '10:05', '10:10', '10:15', '10:20', '10:25']
        y = np.random.randint(s-10, s+10, size=6)

        # Plot with gradient-like fill (using fill_between)
        ax.plot(x, y, color=bg, linewidth=3, marker='o')
        ax.fill_between(x, y, alpha=0.2, color=bg)

        # Customizing the chart to remove "ugly" borders
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)
        ax.spines['left'].set_visible(False)
        ax.spines['bottom'].set_color('#BDC3C7')
        ax.tick_params(axis='x', colors='#7F8C8D')
        ax.tick_params(axis='y', colors='#7F8C8D')
        ax.grid(True, axis='y', linestyle='--', alpha=0.5)

        plt.title('Soil Moisture Trend (Last 30m)', fontsize=10, color='#7F8C8D', loc='left')
        plt.tight_layout()
        plt.show()

btn_refresh.on_click(update_ui)

# Initial Load
update_ui(None)

# --- 4. Assembly ---
dash = widgets.VBox(
    [header_w, kpi_row, status_row, graph_out, btn_refresh, footer_w]
)

display(dash)